**State Boundaries Cleaning**

Using tl_2024_us_state data, let's filter down to desired columns, as well as the desired state. This process can be expanded in the future to include multiple states.

In [11]:
import sys
from pathlib import Path

notebook_path = Path.cwd()
project_root = notebook_path.parent.parent
sys.path.append(str(project_root))

from scripts.io_helpers import read_raw_layer, export_interim
from scripts.data_helpers import to_buffer_crs, buffer_layer
from scripts import data_config as dc

In [12]:
gdf = read_raw_layer("state_boundary_raw")


print(gdf.columns.tolist())
print(gdf.loc[0])

['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'GEOIDFQ', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']
REGION                                                      3
DIVISION                                                    5
STATEFP                                                    54
STATENS                                              01779805
GEOID                                                      54
GEOIDFQ                                           0400000US54
STUSPS                                                     WV
NAME                                            West Virginia
LSAD                                                       00
MTFCC                                                   G4000
FUNCSTAT                                                    A
ALAND                                             62266513826
AWATER                                              488918898
INTPTLAT                   

First, let's filter on the desired state — for now, we're sticking to Colorado.

In [13]:
gdf_co = gdf[gdf['NAME'] == 'Colorado']
gdf_co.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,GEOIDFQ,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
26,4,8,08,01779779,08,0400000US08,CO,Colorado,00,G4000,A,268418973518,1185541418,+38.9937669,-105.5087122,"POLYGON ((-105.15504 36.99526, -105.15543 36.9..."


Next, let's keep only the desired columns. All we really need for now is geometry and name.

In [14]:
gdf_cleaned = gdf_co[['geometry', 'NAME']]
gdf_cleaned.head()

,geometry,NAME
26,"POLYGON ((-105.15504 36.99526, -105.15543 36.9...",Colorado


**Buffering**

When generating a legality layer for a given state, we must take into account distance from features like water and MVUM roads. If you're near the edge of the given state boundary, it's possible that edge cases may arise where water or roads just outside the border aren't accounted for, leading to an inaccurate layer. Let's buffer the state boundary by 300 ft (the minimum distance from MVUM roads for legal camping) to account for these edge cases.

Before buffering, let's project the data to a common CRS.


In [15]:
gdf_projected = to_buffer_crs(gdf_cleaned)

Now, buffer!

In [17]:
gdf_buffered = buffer_layer(gdf_projected, dc.BUFFER_SETTINGS_METERS["road"])
gdf_buffered.head()

,geometry,NAME
26,"POLYGON ((143525.677 4102169.628, 143523.188 4...",Colorado


Now, let's export to the output folder.

In [ ]:
export_interim(gdf_buffered, "state_boundary_buffered", driver="GPKG", verbose=True)

Saved buffered state boundary to:  /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/outputs/state_boundary_buffered/state_boundary_buffered_co.gpkg
